In [48]:
import pandas as pd
import numpy as np

In [49]:
# Load data
match_df = pd.read_csv('compiled_data\match_data')
character_names = pd.read_csv('compiled_data\character_names')

# Merge the data using the character ids
match_df = match_df.merge(character_names, left_on='character', right_on='characterID')

# Replace the 'character' column with the 'name' column
match_df['character'] = match_df['name']

# Drop unnecessary columns
match_df = match_df.drop(columns=['Unnamed: 0', 'characterID', 'name'])

match_df.head()

,character,date,game_mode,league,match_id,ranked,win
0,Lucie,2018-01-01 00:00:03,3V3,1,18E42344815440E3BC148E93658B0E2C,False,False
1,Lucie,2018-01-01 00:00:05,2V2,3,7AF171370D804737BEC036A41A369BBF,False,False
2,Lucie,2018-01-01 00:00:11,2V2,0,8CB6BA4D019E4FAC9D713BAA1D0C20B2,False,False
3,Lucie,2018-01-01 00:00:11,2V2,3,E7F34F0CF2694C7597B973FAF2D31E03,False,True
4,Lucie,2018-01-01 00:00:21,3V3,1,210B90DBDD6A4A23AEC467BDEC6C6980,True,False


In [50]:
# Find all 2v2 matches
all_2v2 = match_df.loc[match_df.game_mode == '2V2']

# Find all 3v3 matches
all_3v3 = match_df.loc[match_df.game_mode == '3V3']

'''Casual matches'''
# Find all casual matches
casual = match_df.loc[match_df.ranked == False]

# Find 2v2 casual matches
casual2v2 = casual.loc[casual.game_mode == '2V2']

# Find 3v3 casual matches
casual3v3 = casual.loc[casual.game_mode == '3V3']

'''Ranked matches'''
# Find all ranked matches
ranked = match_df.loc[match_df.ranked == True]

# Find 2v2 ranked matches
ranked2v2 = ranked.loc[ranked.game_mode == '2V2']

# Find 3v3 ranked matches
ranked3v3 = ranked.loc[ranked.game_mode == '3V3']

In [51]:
def win_rates(df, format_str, type_str):
    '''
    Returns the win rates of the input DataFrame
    '''
    if format_str != 'league':
        # Find the win rates
        temp = df.groupby('character').win.sum() / df.groupby('character').win.count()
        
        # Rename 'win' column to 'win_rate'
        win_rates = pd.DataFrame(temp).reset_index().rename(columns={'win':'win_rate'})
        
        # Add 'format' and 'type' column
        win_rates['format'] = format_str
        win_rates['type'] = type_str
    else:
        # Find the ranked pick rates by league
        temp = df.groupby(['character', 'league']).win.sum() / df.groupby(['character', 'league']).win.count()

        # Unstack groupby object
        win_rates = temp.unstack()

        # Rename columns
        rename = {}
        leagues = ['bronze', 'silver', 'gold', 'platinum', 'diamond', 'champion', 'grand_champ']
        for i in range(7):
            rename[i] = leagues[i]
        win_rates = win_rates.rename(columns=rename)
        
        # Replace null values with zeros
        win_rates = win_rates.replace(np.nan, 0)
        
        # Stack column names, and reset index
        win_rates = win_rates.stack().reset_index()
        
        # Rename 'league' column to 'format',
        # label the 'win_rate' column,
        # and add 'type' column
        win_rates = win_rates.rename(columns={'league':'format', 0:'win_rate'})
        win_rates['type'] = type_str
    
    return win_rates

In [52]:
'''Overall win rates'''
# Find the overall win rates
overall_win_rates = win_rates(match_df, 'overall', 'overall')

# Find the overall 2v2 win rates
overall_win_rates_2v2 = win_rates(all_2v2, 'overall', '2v2')

# Find the overall 3v3 win rates
overall_win_rates_3v3 = win_rates(all_3v3, 'overall', '3v3')

# Concatenate into one DataFrame
overall_win_rates = pd.concat([overall_win_rates, overall_win_rates_2v2, overall_win_rates_3v3], ignore_index=True)

'''Casual win rates'''
# Find the casual win rates
casual_win_rates = win_rates(casual, 'casual', 'overall')

# Find the 2v2 casual win rates
casual_win_rates_2v2 = win_rates(casual2v2, 'casual', '2v2')

# Find the 3v3 casual win rates
casual_win_rates_3v3 = win_rates(casual3v3, 'casual', '2v2')

# Concatenate into one DataFrame
casual_win_rates = pd.concat([casual_win_rates, casual_win_rates_2v2, casual_win_rates_3v3], ignore_index=True)

'''Ranked win rates'''
# Find the ranked win rates
ranked_win_rates = win_rates(ranked, 'ranked', 'overall')

# Find the 2v2 ranked win rates
ranked_win_rates_2v2 = win_rates(ranked2v2, 'ranked', '2v2')

# Find the 3v3 ranked win rates
ranked_win_rates_3v3 = win_rates(ranked3v3, 'ranked', '3v3')

# Concatenate into one DataFrame
ranked_win_rates = pd.concat([ranked_win_rates, ranked_win_rates_2v2, ranked_win_rates_3v3], ignore_index=True)

'''League win rates'''
# Find the ranked win rates by league
league_win_rates = win_rates(ranked, 'league', 'overall')

# Find the ranked win rates by league
league_win_rates_2v2 = win_rates(ranked2v2, 'league', '2v2')

# Find the ranked win rates by league
league_win_rates_3v3 = win_rates(ranked3v3, 'league', '3v3')

# Concatenate into one DataFrame
league_win_rates = pd.concat([league_win_rates, league_win_rates_2v2, league_win_rates_3v3], ignore_index=True)


'''Combine all win rates'''
combined_win_rates = pd.concat([overall_win_rates, casual_win_rates, ranked_win_rates, league_win_rates], ignore_index=True)
combined_win_rates.to_csv('compiled_data\combined_win_rates')

In [53]:
def pick_rates(df, format_str, type_str):
    '''
    Returns the win rates of the input DataFrame
    '''
    if format_str != 'league':
        # Find the overall pick rates
        temp = match_df.groupby('character').win.count() / match_df.win.count()

        # Rename 'win' column to 'pick_rate
        pick_rates = pd.DataFrame(temp).reset_index().rename(columns={'win':'pick_rate'})

        # Add 'format' and 'type' column
        pick_rates['format'] = format_str
        pick_rates['type'] = type_str
    else:
        # Find the ranked pick rates by league
        temp = df.groupby(['character', 'league']).win.count() / df.groupby( 'league').win.count()
        
        # Unstack groupby object
        pick_rates = temp.unstack()
        
        # Rename columns
        rename = {}
        leagues = ['bronze', 'silver', 'gold', 'platinum', 'diamond', 'champion', 'grand_champ']
        for i in range(7):
            rename[i] = leagues[i]
        pick_rates = pick_rates.rename(columns=rename)
        
        # Replace null values with zeros
        pick_rates = pick_rates.replace(np.nan, 0)
        
        # Stack column names, and reset index
        pick_rates = pick_rates.stack().reset_index()
        
        # Rename 'league' column to 'format',
        # label the 'pick_rate' column,
        # and add 'type' column
        pick_rates = pick_rates.rename(columns={'league':'format', 0:'pick_rate'})
        pick_rates['type'] = type_str
    
    return pick_rates

In [54]:
'''Overall pick rates'''
# Find the overall pick rates
overall_pick_rates = pick_rates(match_df, 'overall', 'overall')

# Find the overall 2v2 pick rates
overall_pick_rates_2v2 = pick_rates(all_2v2, 'overall', '2v2')

# Find the overall 3v3 pick rates
overall_pick_rates_3v3 = pick_rates(all_3v3, 'overall', '3v3')

# Concatenate into one DataFrame
overall_pick_rates = pd.concat([overall_pick_rates, overall_pick_rates_2v2, overall_pick_rates_3v3], ignore_index=True)

'''Casual pick rates'''
# Find the casual pick rates
casual_pick_rates = pick_rates(casual, 'casual', 'overall')

# Find the 2v2 casual pick rates
casual_pick_rates_2v2 = pick_rates(casual2v2, 'casual', '2v2')

# Find the 3v3 casual pick rates
casual_pick_rates_3v3 = pick_rates(casual3v3, 'casual', '2v2')

# Concatenate into one DataFrame
casual_pick_rates = pd.concat([casual_pick_rates, casual_pick_rates_2v2, casual_pick_rates_3v3], ignore_index=True)

'''Ranked pick rates'''
# Find the ranked pick rates
ranked_pick_rates = pick_rates(ranked, 'ranked', 'overall')

# Find the 2v2 ranked pick rates
ranked_pick_rates_2v2 = pick_rates(ranked2v2, 'ranked', '2v2')

# Find the 3v3 ranked pick rates
ranked_pick_rates_3v3 = pick_rates(ranked3v3, 'ranked', '3v3')

# Concatenate into one DataFrame
ranked_pick_rates = pd.concat([ranked_pick_rates, ranked_pick_rates_2v2, ranked_pick_rates_3v3], ignore_index=True)

'''League pick rates'''
# Find the ranked pick rates by league
league_pick_rates = pick_rates(ranked, 'league', 'overall')

# Find the ranked pick rates by league
league_pick_rates_2v2 = pick_rates(ranked2v2, 'league', '2v2')

# Find the ranked pick rates by league
league_pick_rates_3v3 = pick_rates(ranked3v3, 'league', '3v3')

# Concatenate into one DataFrame
league_pick_rates = pd.concat([league_pick_rates, league_pick_rates_2v2, league_pick_rates_3v3], ignore_index=True)


'''Combine all pick rates'''
combined_pick_rates = pd.concat([overall_pick_rates, casual_pick_rates, ranked_pick_rates, league_pick_rates], ignore_index=True)
combined_pick_rates.to_csv('compiled_data\combined_pick_rates')

In [173]:
# Group by 'character', 'type', and 'format and get pick rates
win_rates_table = combined_win_rates.groupby(['character', 'type', 'format']).agg({'win_rate':'unique'})
# Get pick rate values
win_rates_table = win_rates_table.win_rate.apply(lambda x: x[0])
# Convert to table for easy viewing
win_rates_table = win_rates_table.unstack(level=2)
# Rearrange columns
columns = ['overall', 'casual', 'ranked', 'bronze', 'silver', 'gold', 'platinum', 'diamond', 'champion', 'grand_champ']
win_rates_table = win_rates_table.loc[:, columns]
# Add 'win_rate' level to the columns
win_rates_table.columns = pd.MultiIndex.from_product([['win_rate'], win_rates_table.columns], names=['stat', 'format'])

temp.head()

stat               win_rate                                                    \
format              overall    casual    ranked    bronze    silver      gold   
character type                                                                  
Alysia    2v2      0.508108  0.522936  0.486842  0.500000  0.565217  0.419355   
          3v3      0.481818       NaN  0.425000  0.400000  0.500000  0.307692   
          overall  0.498305  0.519553  0.465517  0.454545  0.538462  0.386364   
Ashka     2v2      0.501976  0.497326  0.515152  0.642857  0.320000  0.750000   
          3v3      0.488263       NaN  0.528302  0.142857  0.521739  0.692308   

stat                                                        
format             platinum   diamond champion grand_champ  
character type                                              
Alysia    2v2      0.666667  0.000000      0.0         0.0  
          3v3      0.500000  0.500000      0.0         NaN  
          overall  0.625000  0.200000      0.0         0.0  
Ashka     2v2      0.500000  0.666667      0.0         0.0  
          3v3      0.714286  0.333333      0.0         NaN

In [174]:
# Group by 'character', 'type', and 'format and get pick rates
pick_rates_table = combined_pick_rates.groupby(['character', 'type', 'format']).agg({'pick_rate':'unique'})
# Get pick rate values
pick_rates_table = pick_rates_table.pick_rate.apply(lambda x: x[0])
# Convert to table for easy viewing
pick_rates_table = pick_rates_table.unstack(level=2)
# Rearrange columns
columns = ['overall', 'casual', 'ranked', 'bronze', 'silver', 'gold', 'platinum', 'diamond', 'champion', 'grand_champ']
pick_rates_table = pick_rates_table.loc[:, columns]
# Add 'pick_rate' level to the columns
pick_rates_table.columns = pd.MultiIndex.from_product([['pick_rate'], pick_rates_table.columns], names=['stat', 'format'])

temp.head()

stat               win_rate                                                    \
format              overall    casual    ranked    bronze    silver      gold   
character type                                                                  
Alysia    2v2      0.508108  0.522936  0.486842  0.500000  0.565217  0.419355   
          3v3      0.481818       NaN  0.425000  0.400000  0.500000  0.307692   
          overall  0.498305  0.519553  0.465517  0.454545  0.538462  0.386364   
Ashka     2v2      0.501976  0.497326  0.515152  0.642857  0.320000  0.750000   
          3v3      0.488263       NaN  0.528302  0.142857  0.521739  0.692308   

stat                                                        
format             platinum   diamond champion grand_champ  
character type                                              
Alysia    2v2      0.666667  0.000000      0.0         0.0  
          3v3      0.500000  0.500000      0.0         NaN  
          overall  0.625000  0.200000      0.0         0.0  
Ashka     2v2      0.500000  0.666667      0.0         0.0  
          3v3      0.714286  0.333333      0.0         NaN

In [178]:
statistics_table = win_rates_table.merge(pick_rates_table, left_index=True, right_index=True)
statistics_table.to_csv('compiled_data/statistics_table')

statistics_table

stat               win_rate                                                    \
format              overall    casual    ranked    bronze    silver      gold   
character type                                                                  
Alysia    2v2      0.508108  0.522936  0.486842  0.500000  0.565217  0.419355   
          3v3      0.481818       NaN  0.425000  0.400000  0.500000  0.307692   
          overall  0.498305  0.519553  0.465517  0.454545  0.538462  0.386364   
Ashka     2v2      0.501976  0.497326  0.515152  0.642857  0.320000  0.750000   
          3v3      0.488263       NaN  0.528302  0.142857  0.521739  0.692308   
          overall  0.495708  0.487032  0.521008  0.476190  0.416667  0.720000   
Bakko     2v2      0.481865  0.477876  0.487500  0.100000  0.500000  0.555556   
          3v3      0.559633       NaN  0.517241  0.666667  0.600000  0.571429   
          overall  0.509934  0.518135  0.495413  0.230769  0.533333  0.558824   
Blossom   2v2      0.492063  0.553571  0.402597  0.307692  0.500000  0.521739   
          3v3      0.559701       NaN  0.527778  0.333333  0.615385  0.500000   
          overall  0.520124  0.561905  0.442478  0.312500  0.548387  0.513514   
Croak     2v2      0.500000  0.503185  0.496552  0.470588  0.516667  0.485714   
          3v3      0.436090       NaN  0.465116  0.250000  0.529412  0.400000   
          overall  0.480460  0.473684  0.489362  0.428571  0.519481  0.466667   
Destiny   2v2      0.536232  0.534884  0.538462  0.400000  0.444444  0.611111   
          3v3      0.523810       NaN  0.375000  0.500000  0.285714  0.500000   
          overall  0.531532  0.554795  0.486842  0.428571  0.400000  0.576923   
Ezmo      2v2      0.509434  0.521127  0.485714  1.000000  0.727273  0.285714   
          3v3      0.525773       NaN  0.500000  0.333333  0.625000  0.250000   
          overall  0.517241  0.528169  0.491803  0.500000  0.684211  0.277778   
Freya     2v2      0.463964  0.475177  0.444444  0.454545  0.413793  0.444444   
          3v3      0.500000       NaN  0.500000  0.400000  0.714286  0.400000   
          overall  0.477011  0.484444  0.463415  0.437500  0.511628  0.428571   
Iva       2v2      0.559006  0.531532  0.620000  0.555556  0.700000  0.571429   
          3v3      0.476190       NaN  0.375000  1.000000  0.222222  0.500000   
          overall  0.530612  0.519553  0.560606  0.600000  0.551724  0.555556   
Jade      2v2      0.485039  0.501149  0.450000  0.368421  0.373333  0.535714   
          3v3      0.531507       NaN  0.567010  0.333333  0.628571  0.518519   
          overall  0.502000  0.507824  0.488215  0.360000  0.454545  0.530120   
...                     ...       ...       ...       ...       ...       ...   
Raigon    2v2      0.497573  0.520325  0.463855  0.466667  0.400000  0.509434   
          3v3      0.511848       NaN  0.515625  0.500000  0.428571  0.578947   
          overall  0.502408  0.516539  0.478261  0.477273  0.406977  0.527778   
Rook      2v2      0.500000  0.500000  0.500000  0.250000  0.535714  0.681818   
          3v3      0.493421       NaN  0.560976  0.833333  0.444444  0.500000   
          overall  0.497222  0.485232  0.520325  0.500000  0.500000  0.633333   
Ruh Kaan  2v2      0.465909  0.442623  0.518519  0.416667  0.375000  0.647059   
          3v3      0.473118       NaN  0.526316  0.250000  0.833333  0.250000   
          overall  0.468401  0.448980  0.520548  0.375000  0.500000  0.571429   
Shifu     2v2      0.521739  0.471698  0.589744  0.166667  0.727273  0.612903   
          3v3      0.555556       NaN  0.466667  0.400000  0.714286  0.428571   
          overall  0.533569  0.520000  0.555556  0.272727  0.724138  0.578947   
Sirius    2v2      0.568571  0.540541  0.600000  0.625000  0.526316  0.673077   
          3v3      0.541176       NaN  0.585366  0.250000  0.736842  0.555556   
          overall  0.559615  0.535032  0.597087  0.571429  0.578947  0.655738   
Taya      2v2      0.346667  0.